# Neptune + Optuna

## Before you start

### Install dependencies

In [ ]:
! pip install --quiet optuna neptune-client neptune-contrib['monitoring']

### Import libraries

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

### Create a sample `objective` function for Optuna

In [ ]:
def objective(trial):
   
   data, target = load_breast_cancer(return_X_y=True)
   train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
   dtrain = lgb.Dataset(train_x, label=train_y)

   param = {
      'objective': 'binary',
      'metric': 'binary_logloss',
      'num_leaves': trial.suggest_int('num_leaves', 2, 256),
      'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
      'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
      'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
   }

   gbm = lgb.train(param, dtrain)
   preds = gbm.predict(test_x)
   accuracy = roc_auc_score(test_y, preds)
   
   return accuracy

### Initialize Neptune

In [ ]:
import neptune

Neptune gives you an option of logging data under a public folder as an anonymous user. This is great when you are just trying out the application and don't have a Neptune account yet.  

---

If you already have a [Neptune account](https://neptune.ai/register), you can create your own experiment and start logging to it using your personal API token. Pass your `username` to the `project_qualified_name` argument of the `neptune.init()` method: `project_qualified_name='YOUR_USERNAME/YOUR_PROJECT_NAME`. If you don't have a project yet, keep `/sandbox` at the end. The `sandbox` project is automatically created for you.

In [ ]:
neptune.init(api_token='ANONYMOUS', project_qualified_name='shared/optuna-integration')

## Quickstart

### Step 1: Create an Experiment

This creates an experiment in Neptune.

Once you have a live experiment you can log things to it. 

In [ ]:
neptune.create_experiment('optuna-sweep')

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

### Step 2: Create the Neptune Callback

In [ ]:
import neptunecontrib.monitoring.optuna as opt_utils

In [ ]:
neptune_callback = opt_utils.NeptuneCallback()

### Step 3: Run Optuna with the Neptune Callback

You can view the logging live in the Neptune tab once Optuna you run the below cell

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, callbacks=[neptune_callback])

### Step 4: Stop logging

When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running ```neptune.stop()```.

If you are running Neptune in regular ```.py``` scripts it will stop automatically when your code stops running.

In [ ]:
neptune.stop()

## Advanced Options

### Log charts and study object during sweep

To log interactive charts from `optuna.visualization` and the `optuna.study` object itself, you can set `log_study=True` and `log_charts=True` while creating the Neptune Callback. 

This will log the charts after every iteration, so they can be explored while the sweep is running.

<font color=yellow>Warning:</font> Depending on the size of the `optuna.study` object and the charts, this might add overhead to the sweep. To avoid this, you can log the study object and charts after the sweep.

In [ ]:
# Create experiment
neptune.create_experiment('optuna-sweep-advanced')

# Create callback to log advanced options during the sweep
neptune_callback = opt_utils.NeptuneCallback(log_study=True, log_charts=True)

# Run Optuna with Neptune Callback
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, callbacks=[neptune_callback])

# Stop logging (only for notebooks)
neptune.stop()

### Log charts and study object after the sweep

If you don't want charts from `optuna.visualizations` and the `optuna.study` object to be updated after each iteration, you can log all of them after the sweep is complete to prevent overhead during the sweep. 

In [ ]:
# Create experiment
neptune.create_experiment('optuna-sweep-advanced')

# Create Neptune callback
neptune_callback = opt_utils.NeptuneCallback()

# Run Optuna with Neptune Callback
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, callbacks=[neptune_callback])

# Log Optuna charts and study object after the sweep is complete
opt_utils.log_study_info(study)

# Stop logging (only for notebooks)
neptune.stop()

## Explore results in the Neptune UI